## Ejemplo K-Means

In [ ]:
import seaborn as sns

# Importamos el Dataset Iris
df = sns.load_dataset("iris")
df

In [ ]:
df["species"].value_counts()

> Supongamos que utilizaremos sólo las variables numéricas... "Species", es de hecho la respuesta correcta (la etiqueta). 

In [ ]:
# Definimos X como una Matriz sin la variable Species.
X = df.drop(columns="species")
X

## Ayuda Visual
Vamos a utilizar PCA para poder reducir las dimensiones a un tamaño el cual podamos visualizar: 2D.

In [ ]:
from sklearn.decomposition import PCA
import pandas as pd

## Esto es sólo una ayuda para poder visualizar datos
# que están en más dimensiones de las que podemos ver.
pca = PCA(n_components=2, random_state=1)
pca_X = pca.fit_transform(X)

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(pca_X[:, 0], pca_X[:, 1])
plt.title("Visualización de Iris en 2D.")
plt.tight_layout()

In [ ]:
## Esta es una función que nos permitirá visualizar nuestras etiquetas en un espacio reducido por PCA.
## Además permite la visualización de los centroides de nuestro proceso...


def pca_viz(pca_X, pca_centroids, labels, title=None, cmap="viridis"):
    plt.scatter(pca_X[:, 0], pca_X[:, 1], c=labels, cmap=cmap)
    plt.scatter(
        pca_centroids[:, 0],
        pca_centroids[:, 1],
        marker="*",
        c="red",
        s=150,
    )
    plt.title(title)

### Implementación de K-Means

In [ ]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=2, n_init=10, random_state=1)
labels = km.fit_predict(X)
centroids = km.cluster_centers_
pca_centroids = pca.transform(centroids)

In [ ]:
pca_viz(
    pca_X,
    pca_centroids,
    labels=labels,
    title="Visualización de K-Means en Iris 2D",
)

### Efecto del Escalamiento en K-Means

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_sc = sc.fit_transform(X)
pca = PCA(n_components=2, random_state=1)
pca_X_sc = pca.fit_transform(X_sc)
km = KMeans(n_clusters=2, n_init=10, random_state=1)
sc_labels = km.fit_predict(X_sc)
centroids = km.cluster_centers_
pca_centroids = pca.transform(centroids)
pca_viz(
    pca_X_sc,
    pca_centroids,
    sc_labels,
    title="K-Means de Iris en 2D luego de Estandarizar los datos. ",
)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

mm = MinMaxScaler()
X_mm = mm.fit_transform(X)
pca = PCA(n_components=2, random_state=1)
pca_X_mm = pca.fit_transform(X_mm)
km = KMeans(n_clusters=3, n_init=10, random_state=1)
mm_labels = km.fit_predict(X_mm)
centroids = km.cluster_centers_
pca_centroids = pca.transform(centroids)

pca_viz(
    pca_X_mm,
    pca_centroids,
    mm_labels,
    title="K-Means de Iris en 2D luego de Normalizar los datos.",
)

## Ejemplo más avanzado sin entrenar con todos los datos...

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(X, test_size=0.25, random_state=1)

> Estamos dejando un 25% de los datos fuera para poder chequear cuál sería la `predicción` que se le dan a dichos datos.

In [ ]:
pca = PCA(n_components=2)
km = KMeans(n_clusters=2, n_init=10)
sc = StandardScaler()
## Fit siempre se hace con datos de `Entrenamiento`.

## Escalamos los datos...
sc.fit(X_train)
X_train_sc = sc.transform(X_train)
X_test_sc = sc.transform(X_test)

# Generamos las coordenadas del PCA para visualizar
pca.fit(X_train_sc)
pca_train = pca.transform(X_train_sc)
pca_test = pca.transform(X_test_sc)

train_labels = km.fit_predict(X_train_sc)
test_labels = km.predict(X_test_sc)
centroids = km.cluster_centers_
pca_centroids = pca.transform(centroids)

pca_viz(pca_train, pca_centroids, train_labels)
pca_viz(pca_test, pca_centroids, test_labels, cmap="tab20b")

## Cuál es el K óptimo?

In [ ]:
def elbow_curve(X, k_max=10, color="blue", title=None):
    wc = []
    for k in range(1, k_max + 1):
        km = KMeans(n_clusters=k, random_state=1)
        km.fit(X)
        wc.append(km.inertia_)

    k = [*range(1, k_max + 1)]
    plt.plot(k, wc, c=color, marker="*")
    plt.title(title)
    plt.xlabel("Número de Clústers")
    plt.ylabel("Within Distance")
    return wc


wc = elbow_curve(
    X_train,
    k_max=15,
    color="red",
    title="Curva del Codo para el Dataset Iris, sólo con Train Set.",
)

In [ ]:
wc